In [78]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
%matplotlib inline
from matplotlib.pylab import rcParams
from sklearn.metrics.pairwise import manhattan_distances
from scipy.spatial import distance

In [79]:
#Quarter 1 and 2
stations_2014_Q1_2 = pd.read_csv('Datasets/Divvy_Stations_Trips_2014_Q1Q2//Divvy_Stations_2014-Q1Q2.csv')
dataset_2014_Q1_2 = pd.read_csv('Datasets/Divvy_Stations_Trips_2014_Q1Q2/Divvy_Trips_2014_Q1Q2.csv')
#Quarter 3 
stations_2014_Q3_4  = pd.read_csv('Datasets/Divvy_Stations_Trips_2014_Q3Q4/Divvy_Stations_2014-Q3Q4.csv')
dataset_2014_07  = pd.read_csv('Datasets/Divvy_Stations_Trips_2014_Q3Q4/Divvy_Trips_2014-Q3-07.csv')
dataset_2014_08_09  = pd.read_csv('Datasets/Divvy_Stations_Trips_2014_Q3Q4/Divvy_Trips_2014-Q3-0809.csv')
dataset_2014_Q3 = pd.concat([dataset_2014_07, dataset_2014_08_09])
#Quarter 4 
dataset_2014_Q4 = pd.read_csv('Datasets/Divvy_Stations_Trips_2014_Q3Q4/Divvy_Trips_2014-Q4.csv')

In [80]:
dataset_2014_full=pd.concat([dataset_2014_Q1_2,dataset_2014_Q3,dataset_2014_Q4])
dataset_2014 = dataset_2014_full[:1000]

In [81]:
dataset_2014.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,2355134,6/30/2014 23:57,7/1/2014 0:07,2006,604,131,Lincoln Ave & Belmont Ave,303,Broadway & Cornelia Ave,Subscriber,Male,1988.0
1,2355133,6/30/2014 23:56,7/1/2014 0:00,2217,263,282,Halsted St & Maxwell St,22,May St & Taylor St,Subscriber,Male,1992.0
2,2355130,6/30/2014 23:33,6/30/2014 23:35,2798,126,327,Sheffield Ave & Webster Ave,225,Halsted St & Dickens Ave,Subscriber,Male,1993.0
3,2355129,6/30/2014 23:26,7/1/2014 0:24,173,3481,134,Peoria St & Jackson Blvd,194,State St & Wacker Dr,Subscriber,Female,1988.0
4,2355128,6/30/2014 23:16,6/30/2014 23:26,173,638,320,Loomis St & Lexington St,134,Peoria St & Jackson Blvd,Subscriber,Female,1988.0


In [82]:
#Features about UserType
usertype=[]
for user in dataset_2014.usertype:
    if user=='Customer':
        usertype.append(-1)
    elif user=='Subscriber':
        usertype.append(1)
dataset_2014['usertype']=usertype
#Gender male =1, female =-1
gendertype = []
for gender in dataset_2014.gender:
    if gender=='Male':
        gendertype.append(1)
    elif gender=='Female':
        gendertype.append(-1)
    else:
        gendertype.append(0)
dataset_2014['gender']=gendertype
#Age
age=[]
for user,year in zip(dataset_2014.usertype,dataset_2014.birthyear):
    if user ==-1:
        age.append(0)
    elif user==1:
        age.append(2014-year)
dataset_2014['age']=age
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [83]:
# Features about departure time 
dataset_2014['starttime'] = pd.to_datetime(dataset_2014['starttime'],infer_datetime_format=True)
dataset_2014['month'] = dataset_2014['starttime'].dt.month

day=[]
for weekday in dataset_2014['starttime'].dt.weekday_name:
    if weekday=='Sunday':
        day.append(0)
    else:
        day.append(1)
dataset_2014['weekday'] = day
dataset_2014['hour'] = dataset_2014['starttime'].dt.hour

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [84]:
#Features about Stations
stations_2014_Q1_2.dtypes
    

id               int64
name            object
latitude       float64
longitude      float64
dpcapacity       int64
online date     object
dtype: object

In [85]:
dataset_2014['station_pair']=list(zip(dataset_2014.from_station_id,dataset_2014.to_station_id))
#stations_2014_Q1_2['lat_lon']=list(zip(sta))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
geography = []
geography_o = []
geography_d = []
man_dist = []
for origin,destination in zip(dataset_2014.from_station_id,dataset_2014.to_station_id):
    org = stations_2014_Q1_2.loc[stations_2014_Q1_2['id'] == origin]
    dest = stations_2014_Q1_2.loc[stations_2014_Q1_2['id'] == destination]
    
    tup = (org.latitude.item(),org.longitude.item(),dest.latitude.item(),dest.longitude.item())
    
    geography.append(tup)
    man_dist.append(distance.cityblock([org.latitude.item(),org.longitude.item()],[dest.latitude.item(),dest.longitude.item()]))
dataset_2014['geo'] = geography
dataset_2014['man_dist'] = man_dist

#dataset_2014.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [154]:
duration_dataset = dataset_2014[['usertype','gender','age','month','weekday','hour','man_dist','tripduration']]
duration_dataset['tripduration'] = np.log10(1+duration_dataset.tripduration)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [156]:
#duration_dataset.set_index('station_pair',drop=False)

duration_dataset['tripduration']

0      2.781755
1      2.421604
2      2.103804
3      3.541829
4      2.805501
         ...   
995    2.700704
996    2.761176
997    2.812245
998    2.745075
999    2.600973
Name: tripduration, Length: 1000, dtype: float64

In [157]:
X = duration_dataset.iloc[:,:-1]
y = duration_dataset.iloc[:,-1]


In [171]:
# Import necessary modules
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=0)

clf = linear_model.Lasso(alpha=0.0001)

# Fit the regressor to the data
clf.fit(X_train, y_train)


# Predict on the test data: y_pred
y_pred = clf.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2: {}".format(r2_score(y_test,y_pred)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("MEan Absolute error ",mean_absolute_error(y_test,y_pred))
print("Root Mean Squared Error: {}".format(rmse))

R^2: 0.49464647873533063
MEan Absolute error  0.14935065078372248
Root Mean Squared Error: 0.22199786266244942


In [185]:
#XGBOOST
import xgboost as xgb

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 3, 
                          reg_alpha = 0.001, n_estimators = 10)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

print("R^2: {}".format(r2_score(y_test,preds)))

print("MEan Absolute error ",mean_absolute_error(y_test,preds))


[18:43:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R^2: -6.166097899189195
MEan Absolute error  0.7912803365775227


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
